In [ ]:
# pths = [r'\\169.254.138.20\Andre\data\PDAC Purity Project\PDAC PURITY Final Analysis\LCM_H&Es_10x\czi\ometiff_20x']
pths=[r'\\10.99.68.52\Kiemendata\PanIN phenotyping\PDAC Purity Project\LCM H&Es\try 20x AK']
# WSIs = [glob(f'{pth}\*.ome.tiff') for pth in pths]
WSIs = [glob(f'{pth}\*.tif') for pth in pths]


In [ ]:
WSIs[0]


In [ ]:
img_pths = [img_pth for p, img_pth in enumerate(WSIs[0])]
img = imread(img_pths[0])

In [ ]:
img = img/255
img.shape

In [ ]:
import json
from stardist.models import Config2D, StarDist2D

def load_model(model_path: str) -> StarDist2D:
    # Load StarDist model weights, configurations, and thresholds
    with open(model_path + '\\config.json', 'r') as f:
        config = json.load(f)
    with open(model_path + '\\thresholds.json', 'r') as f:
        thresh = json.load(f)
    model = StarDist2D(config=Config2D(**config), basedir=model_path, name='offshoot_model')
    model.thresholds = thresh
    print('Overriding defaults:', model.thresholds, '\n')
    model.load_weights(model_path + '\\weights_best.h5')
    return model
# model = load_model(r"\\10.99.68.178\andreex\data\Stardist\qupath_training_annotations\models\fallopian_tube_12_7_2023_lr_0.001_epochs_400_pt_40")
model = load_model(r"\\10.99.68.178\andreex\data\Stardist\12_12_hyperparameter_models\monkey_12_12_2023_lr_0.001_epochs_400_pt_10_gaus_ratio_0")

In [ ]:
labels, polys = model.predict_instances_big(img, axes='YXC', block_size=4096, min_overlap=128, context=128, n_tiles=(4,4,1))


In [ ]:
tile_size = 512*50

y_cor = 15000
y_cor_end = y_cor + tile_size
x_cor = 10000
x_cor_end = x_cor + tile_size

cropped_img = img[y_cor:y_cor_end, x_cor:x_cor_end]
cropped_lbl = labels[y_cor:y_cor_end, x_cor:x_cor_end]

lbl_cmap = random_label_cmap()

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(cropped_img)
plt.imshow(cropped_lbl, cmap=lbl_cmap, alpha=0.25)
plt.axis('off')
plt.show()